# user_reviews

In [1]:
from extract import jsongz

#df_original = jsongz.leer_archivo("..\\data\\raw\\user_reviews.json.gz")
df_original = jsongz.leer_archivo("../data/raw/user_reviews.json.gz")

In [2]:
df = df_original.copy()
df.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [4]:
df.shape[0]

25799

El dataframe contiene: 25799 registros

Evaluar los tipos de datos utilizados en las columnas del dataframe

In [5]:
from transform import info

info.mostrar_info_key_types(df)

user_id - {<class 'str'>}
user_url - {<class 'str'>}
reviews - {<class 'list'>}


Recordar que las columnas que contiene valores iguales a "float", podria indica que contine valores nulos.

La columna "reviews" utiliza datos de tipo "list". Hay que desanidarlo.

In [6]:
from transform import normalizar

df = normalizar.desanidar_dataframe(df, ["reviews"])

df.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."


Volver a revisar los tipos de datos, para asegurar que no se utilizen estructuras de datos

In [7]:
from transform import info

info.mostrar_info_key_types(df)

user_id - {<class 'str'>}
user_url - {<class 'str'>}
reviews - {<class 'float'>, <class 'dict'>}


Ahora, la columna "reviews" contiene valores de tipo "dict". Hay que expandirlo.

In [8]:
from transform import normalizar

df = normalizar.expandir_columna(df, "reviews")

df.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


Volver a revisar los tipos de datos, para asegurar que no se utilizen estructuras de datos

In [9]:
from transform import info

info.mostrar_info_key_types(df)

user_id - {<class 'str'>}
user_url - {<class 'str'>}
funny - {<class 'str'>, <class 'float'>}
posted - {<class 'str'>, <class 'float'>}
last_edited - {<class 'str'>, <class 'float'>}
item_id - {<class 'str'>, <class 'float'>}
helpful - {<class 'str'>, <class 'float'>}
recommend - {<class 'bool'>, <class 'float'>}
review - {<class 'str'>, <class 'float'>}


Se elimina las columnas que no son relevantes para los objetivos del proyecto. Esto está desarrollado en una hoja de cálculo, en una matriz de objetivos vs. datos.

https://docs.google.com/spreadsheets/d/1ccOxyOM_yctvpk4-CbpX4kCihNn1rX1QifKjbtov10s/edit?usp=sharing

In [11]:
from transform import limpiar

limpiar.eliminar_columnas_innecesarias(
    df,
    [
        "user_id",
        "user_url",
        "funny",
        "last_edited",
        "helpful",
    ],
)
df.head()

,posted,item_id,recommend,review
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


In [10]:
df.shape[0]

59333

Ahora, la cantidad de filas en el dataframe es de: 59333

## Variables
Las que utiliza este daframe son:

### Variables temporales
* posted - variable temporal

### Variables categóricas
* item_id - Es una variable categórica nominal (identificador)
* recommend - variable categórica nominal
* review - variable categóricas nominal

## Manejando la columna "posted"

Cambiaremos el formato de la fecha en "posted"

In [10]:
from datetime import datetime

fecha = "Posted November 5, 2011."

def formatear_fecha(fecha):
    try :
        fecha = datetime.strptime(str(fecha), "Posted %B %d, %Y.")
    except Exception as Error:
        None
    return fecha

df['posted'] = df['posted'].apply(formatear_fecha)

Revisamos la cantidad de registros de fecha con un formato no válido.

In [11]:
df_copy = df.copy()

from datetime import datetime

df_copy = df_copy[df_copy["posted"].apply(lambda x: not isinstance(x, datetime))]

conteo_por_valor = df_copy["posted"].value_counts().to_dict()

print(f"La cantidad de valores diferentes son: {len(conteo_por_valor)}")
df_copy["posted"].value_counts().to_dict()

La cantidad de valores diferentes son: 262


{'Posted June 25.': 83,
 'Posted January 5.': 81,
 'Posted January 3.': 78,
 'Posted August 12.': 72,
 'Posted January 4.': 71,
 'Posted June 24.': 67,
 'Posted August 13.': 65,
 'Posted February 5.': 65,
 'Posted June 26.': 65,
 'Posted January 1.': 64,
 'Posted July 1.': 64,
 'Posted July 4.': 63,
 'Posted May 14.': 62,
 'Posted January 2.': 61,
 'Posted January 22.': 60,
 'Posted July 3.': 60,
 'Posted January 11.': 59,
 'Posted January 10.': 59,
 'Posted March 25.': 59,
 'Posted July 8.': 58,
 'Posted February 26.': 58,
 'Posted January 23.': 58,
 'Posted June 29.': 57,
 'Posted January 16.': 57,
 'Posted April 30.': 56,
 'Posted July 28.': 56,
 'Posted July 10.': 56,
 'Posted March 11.': 55,
 'Posted February 6.': 55,
 'Posted April 12.': 54,
 'Posted January 14.': 54,
 'Posted May 28.': 54,
 'Posted January 20.': 54,
 'Posted January 30.': 54,
 'Posted January 15.': 53,
 'Posted June 27.': 53,
 'Posted January 17.': 53,
 'Posted September 2.': 53,
 'Posted February 13.': 53,
 'Po

Debería continuar analizando los valores de esta columna, pero no dejaré. No son demasiados registros.

Los registros que no son validos los reemplazo por NaT

In [12]:
from datetime import datetime

def reemplazar_fechas_no_validas_por_NaT(x):
    return float('nan') if not isinstance(x, datetime) else x

df["posted"] = df["posted"].apply(lambda x: reemplazar_fechas_no_validas_por_NaT(x))

Verifico la cantidad de registros en "posted" no válidos.

In [13]:
df_copy = df.copy()

from datetime import datetime

df_copy = df_copy[df_copy["posted"].apply(lambda x: not isinstance(x, datetime))]

conteo_por_valor = df_copy["posted"].value_counts().to_dict()

print(f"La cantidad de valores no validos para fecha, diferentes entre si, son: {len(conteo_por_valor)}")
df_copy["posted"].value_counts().to_dict()

La cantidad de valores no validos para fecha, diferentes entre si, son: 0


{}

## Manejando la columna "review"

Primero, reviso los valores diferentes a "str" en la columna.

In [8]:
from transform import info
info.mostrar_info_key_types(df)

posted - {<class 'str'>, <class 'float'>}
item_id - {<class 'str'>, <class 'float'>}
recommend - {<class 'bool'>, <class 'float'>}
review - {<class 'str'>, <class 'float'>}


In [14]:
df_copy = df.copy()

column = "review"
tod = str

df_copy = df_copy[df_copy[column].apply(lambda x: not isinstance(x, tod))]

conteo_por_valor = df_copy[column].value_counts().to_dict()

print(f"La cantidad de valores no validos, diferentes entre si, son: {len(conteo_por_valor)}")
df_copy[column].value_counts().to_dict()

La cantidad de valores no validos, diferentes entre si, son: 0


{}

Como la cantidad de valores no validos, diferentes entre si, es 0. Eso me inidca que los valores de tipo "float" que aparecen al momento de listar los tipos de dato en cada columna, son los registros nulos.

Ahora, se requiere cambiar el contenido de la columna "review" a valores que simplificanran saber si es un comentario positivo, negativo o neutro.

In [15]:
from textblob import TextBlob

def evaluar_review(x):
    if not isinstance(x, str):
        return 0
    else:
        sentiment = TextBlob(x)
        if sentiment.polarity > 0:
            return 1
        elif sentiment.polarity < 0:
            return -1
        else:
            return 0
df["review"] = df["review"].apply(lambda x: evaluar_review(x))

df.head()

,posted,item_id,recommend,review
0,2011-11-05,1250,True,1
1,2011-07-15,22200,True,1
2,2011-04-21,43110,True,1
3,2014-06-24,251610,True,1
4,2013-09-08,227300,True,-1


## Limpieza

Revisando la cantidad de celdas vacias por columna.

In [16]:
from transform import info

info.celdas_vacias(df)

,Cantidad,Porcentaje
posted,10147,0.171018
item_id,28,0.000472
recommend,28,0.000472
review,0,0.000000


Como la columna "item_id" es un identificador, no deberia tener valores nulos

In [17]:
from transform import limpiar

limpiar.eliminar_filas_por_celdas_vacias_en(df, ["item_id"])

la cantidad de filas en el dataframe es: 59333
despues de eliminar las filas vacias es: 59333


In [ ]:
from transform import limpiar

limpiar.eliminar_filas_vacias(df)

In [18]:
from transform import limpiar

df = limpiar.eliminar_filas_duplicadas(df)

la cantidad de filas en el dataframe es: 59333
despues de eliminar las filas repetidas es: 44301


,posted,item_id,recommend,review
0,2011-11-05,1250,True,1
1,2011-07-15,22200,True,1
2,2011-04-21,43110,True,1
3,2014-06-24,251610,True,1
4,2013-09-08,227300,True,-1
...,...,...,...,...
59305,NaT,282660,True,1
59309,NaT,220090,True,-1
59310,NaT,262850,True,-1
59313,NaT,298110,False,0


Volvemos a revisar las celdas vacias

In [ ]:
from transform import info

info.celdas_vacias(df)

In [12]:
df.shape[0]

59333

Ahora, la cantidad de filas en el dataframe es de: 

## Load

In [13]:
df.to_parquet("../data/intermediate/users_reviews.parquet")